In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# import tensorflow as tf
# gpus = tf.config.list_physical_devices('GPU')
# gpu = gpus[0]
# tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import pickle
from collections import Counter

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
from razdel import tokenize
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import PorterStemmer
import spacy ### не работает нормально совместно с tf gpu + подгружать до tf
from spacy import displacy

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [4]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, LSTM

### get files

In [5]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# drive = GoogleDrive(gauth)

In [6]:
# downloaded = drive.CreateFile({'id': '1d1-5FwxK53ePwygNWeG7jhsOWZbi5HOv'})
# downloaded.GetContentFile('train_docs.pkl')

In [7]:
# downloaded = drive.CreateFile({'id': '1MMOY477t965G0C5DtXeREVp0X85UaNq5'})
# downloaded.GetContentFile('test_docs.pkl')

In [8]:
# import zipfile
# with zipfile.ZipFile('D:/GeekBrains/nlp/les06/IMDB_sentiment_dataset.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

In [9]:
train_df = pd.read_csv("train.tsv", delimiter="\t", encoding = 'utf8')
test_df = pd.read_csv("test.tsv", delimiter="\t", encoding = 'utf8')

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


In [10]:
train_df.head()

,is_positive,review
0,0,"Dreamgirls, despite its fistful of Tony wins i..."
1,0,This show comes up with interesting locations ...
2,1,I simply love this movie. I also love the Ramo...
3,0,Spoilers ahead if you want to call them that.....
4,1,My all-time favorite movie! I have seen many m...


In [11]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищенный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = re.sub(r'<br />', ' ', text)
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•`'·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub("[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub("\\'", "'", text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", '', text)
    text = re.sub(r'[\xad]', '', text.strip())


    return text

In [12]:
txt = train_df['review'].iloc[5]
txt

"Wonderful film, one of the best horror films of the 70s. She is realistic settings and atmospheres. As usual it was inevitable the usual negative comments. I have noticed that most horror films of a certain period many times fail to reach even sufficiency. Obviously because most horror movies are old and must be denigrati, is like a mental mechanism that moves the minds of the potential of music critics here.<br /><br />Before you read the review already knew what was the final judgment. In the film a good gift because 10 is really well done. Raines reads quite well and the film as a way in which it was produced reminds me a lot of Kubrick films. He really impression. Excellent film really. I consider a film anthology of years'70."

In [13]:
clean_text(txt)

C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Possible nested set at position 41
  
C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 6
  from ipykernel import kernelapp as app


'Wonderful film one of the best horror films of the s She is realistic settings and atmospheres As usual it was inevitable the usual negative comments I have noticed that most horror films of a certain period many times fail to reach even sufficiency Obviously because most horror movies are old and must be denigrati is like a mental mechanism that moves the minds of the potential of music critics here  Before you read the review already knew what was the final judgment In the film a good gift because  is really well done Raines reads quite well and the film as a way in which it was produced reminds me a lot of Kubrick films He really impression Excellent film really I consider a film anthology of years'

In [14]:
train_df['review'] = train_df['review'].apply(lambda text: clean_text(text))
test_df['review'] = test_df['review'].apply(lambda text: clean_text(text))

In [15]:
#@title Начинаем классифицировать! { vertical-output: true, display-mode: "form" }
positive_words = 'love', 'great', 'best', 'wonderful', 'favorite', 'memorable', 'popular', 'hooked'#@param {type:"raw"}
negative_words = 'worst', 'awful', 'crap', 'poor', 'terrible', 'boring', 'dreadful', 'worse', 'confusing', 'bad' #@param {type:"raw"}

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))
# if accuracy > 0.71:
#     from IPython.display import Image, display
#     display(Image('https://s3.amazonaws.com/achgen360/t/rmmoZsub.png', width=500))

Test accuracy = 71.22%


In [16]:
res = {}
res['keywords'] = accuracy

In [17]:
from nltk.corpus import stopwords
stopword_en = stopwords.words('english')
with open('stopwords.txt', encoding='utf8') as f:
    additional_stopwords = set([w.strip() for w in f.readlines() if w]) # from https://github.com/stopwords-iso and others
stopword_en += list(additional_stopwords)
len(stopword_en)

1178

In [18]:
cache = {}
morph = pymorphy2.MorphAnalyzer()
def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_en] # [6]
    words_lem_without_stopwords = ' '.join(words_lem_without_stopwords) #list to string
    
    return words_lem_without_stopwords

In [19]:
cache2 = {}
def stem(text, stemmer):
    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_stem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache2: # [4]
                words_stem.append(cache2[w])
            else: # [5]
                temp_cach = cache2[w] = stemmer.stem(w)
                words_stem.append(temp_cach)
    
    words_stem_without_stopwords=[i for i in words_stem if not i in stopword_en] # [6]
    words_stem_without_stopwords = ' '.join(words_stem_without_stopwords) #list to string
    
    return words_stem_without_stopwords

In [20]:
train_df['review_lem'] = train_df['review'].apply(lambda text: lemmatization(text))
test_df['review_lem'] = test_df['review'].apply(lambda text: lemmatization(text))

In [21]:
def eval_model(model, X, y):
    preds = model.predict(X)
    
    return accuracy_score(y, preds)

In [22]:
def get_accuracy(X_train, y_train, X_test, y_test, classifier, vectorizer):

    model = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', classifier)
    ])

    model.fit(X_train, y_train)
    return eval_model(model, X_test, y_test), classifier, vectorizer

#### count vectorizer

In [23]:
acc, clf, vect = get_accuracy(train_df['review'], train_df['is_positive'], test_df['review'], test_df['is_positive'], LogisticRegression(), CountVectorizer(stop_words = stopword_en, min_df = 0.01))
res['count_vect'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'keywords': 0.71224, 'count_vect': 0.85692}

In [24]:
acc, clf, vect = get_accuracy(train_df['review_lem'], train_df['is_positive'], test_df['review_lem'], test_df['is_positive'], LogisticRegression(), CountVectorizer(stop_words = stopword_en, min_df = 0.01))
res['count_vect_lem'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'keywords': 0.71224, 'count_vect': 0.85692, 'count_vect_lem': 0.85692}

In [25]:
import eli5
eli5.show_weights(clf, vec = vect, top=40)

Weight?,Feature
+1.741,wonderfully
+1.313,funniest
+1.159,subtle
+1.140,delightful
+1.139,rare
+1.104,excellent
+1.019,perfectly
+1.015,finest
+0.994,surprisingly
+0.991,incredible


In [26]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(clf, test_df['review_lem'].iloc[6], vec = vect, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


#### tfidf vectorizer

In [27]:
acc, clf, vect = get_accuracy(train_df['review'], train_df['is_positive'], test_df['review'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666}

с tfidf результат чуть выше

In [28]:
acc, clf, vect = get_accuracy(train_df['review_lem'], train_df['is_positive'], test_df['review_lem'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_lem'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666}

In [29]:
eli5.show_weights(clf, vec = vect, top=40)

Weight?,Feature
+5.854,excellent
+5.600,great
+4.515,best
+4.466,perfect
+4.374,wonderful
+4.292,favorite
+4.013,amazing
+3.484,loved
+3.376,fantastic
+3.356,wonderfully


In [30]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(clf, test_df['review_lem'].iloc[6], vec = vect, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


качество от наличия лемматизации не изменилось

#### word n-grams

In [31]:
acc, clf, vect = get_accuracy(train_df['review'], train_df['is_positive'], test_df['review'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(ngram_range=(1, 2), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_n12'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748}

In [32]:
acc, clf, vect = get_accuracy(train_df['review'], train_df['is_positive'], test_df['review'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(ngram_range=(1, 3), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_n13'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748}

добавление биграмм немного улучшает результат, дальнейшее увеличение ничего не меняет

In [33]:
acc, clf, vect = get_accuracy(train_df['review'], train_df['is_positive'], test_df['review'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(ngram_range=(2, 2), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_n22'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112}

только пары слов (без униграмм) сильно ухудшили результат

#### char n-grams

In [34]:
acc, clf, vect = get_accuracy(train_df['review'], train_df['is_positive'], test_df['review'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(analyzer='char', ngram_range=(2, 6), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_nc26'] = acc
res

{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748}

In [35]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(clf, test_df['review_lem'].iloc[6], vec = vect, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


In [36]:
acc, clf, vect = get_accuracy(train_df['review_lem'], train_df['is_positive'], test_df['review_lem'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(analyzer='char', ngram_range=(2, 6), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_nc26_lem'] = acc
res

{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748,
 'tfidf_vect_nc26_lem': 0.87372}

n-граммы символов хорошо подняли точность, а вот лемматизация в данном случае ее, наоборот, ухудшила

#### stemmer

In [37]:
stemmer = PorterStemmer()

In [38]:
train_df['review_stem'] = train_df['review'].apply(lambda text: stem(text, stemmer))
test_df['review_stem'] = test_df['review'].apply(lambda text: stem(text, stemmer))

In [39]:
acc, clf, vect = get_accuracy(train_df['review_stem'], train_df['is_positive'], test_df['review_stem'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(ngram_range=(1, 1), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_stem'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748,
 'tfidf_vect_nc26_lem': 0.87372,
 'tfidf_vect_stem': 0.86948}

стемминг сработал чуть лучше лемматизации на униграммах

In [40]:
acc, clf, vect = get_accuracy(train_df['review_stem'], train_df['is_positive'], test_df['review_stem'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(ngram_range=(1, 2), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_n12_stem'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748,
 'tfidf_vect_nc26_lem': 0.87372,
 'tfidf_vect_stem': 0.86948,
 'tfidf_vect_n12_stem': 0.87136}

In [41]:
acc, clf, vect = get_accuracy(train_df['review_stem'], train_df['is_positive'], test_df['review_stem'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(analyzer='char', ngram_range=(2, 6), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_nс26_stem'] = acc
res

{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748,
 'tfidf_vect_nc26_lem': 0.87372,
 'tfidf_vect_stem': 0.86948,
 'tfidf_vect_n12_stem': 0.87136,
 'tfidf_vect_nс26_stem': 0.87272}

стемминг не настолько ухудшил результат на символах как лемматизация, но результат все равно ниже

#### NER

In [42]:
# with open('train_docs.pkl', 'rb') as f:
#     train_docs = pickle.load(f)
    
# with open('test_docs.pkl', 'rb') as f:
#     test_docs = pickle.load(f)

In [43]:
nlp = spacy.load('en_core_web_sm', disable=["tagger", "parser", "attribute_ruler", "lemmatizer", "textcat"])

In [44]:
def replace_person(text):
    
    if not isinstance(text, str):
        text = str(text)
    
    doc = nlp(text)
    d = []

    for token in doc:
        if token.ent_type_ == 'PERSON':
            d.append(token.ent_type_)
        else:
            d.append(token.text)
    return ' '.join(d)

In [45]:
replace_person(train_df['review'].iloc[0])

'Dreamgirls despite its fistful of PERSON wins in an incredibly weak year on Broadway has never been what one would call a jewel in the crown of stage musicals However that is not to say that in the right cinematic hands it could not be fleshed out and polished into something worthwhile onscreen Unfortunately what transfers to the screen is basically a slavishly faithful version of the stage hit with all of its inherent weaknesses intact First the score has never been one of the strong points of this production and the film does not change that factor There are lots of songs perhaps too many but few of them are especially memorable The closest any come to catchy tunes are the title song and One Night Only   the much acclaimed And I Am Telling You That I Am Not Going is less a great song than it is a dramatic set piece for the character of PERSON PERSON PERSON The film is slick and technically wellproduced but the story and characters are surprisingly thin and lacking in any resonance T

In [46]:
%%time
train_df['review_ner'] = train_df['review'].apply(lambda text: replace_person(text))
test_df['review_ner'] = test_df['review'].apply(lambda text: replace_person(text))

Wall time: 13min 31s


In [47]:
train_df.to_csv('train_processed.csv')
test_df.to_csv('test_processed.csv')

In [48]:
acc, clf, vect = get_accuracy(train_df['review_ner'], train_df['is_positive'], test_df['review_ner'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(ngram_range=(1, 2), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_n12_ner'] = acc
res

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_ссылка', 'интерфакс', 'как_сообщили_риа_', 'новости', 'передает_риа_', 'сообщает_', 'сообщает_риа_', 'хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748,
 'tfidf_vect_nc26_lem': 0.87372,
 'tfidf_vect_stem': 0.86948,
 'tfidf_vect_n12_stem': 0.87136,
 'tfidf_vect_nс26_stem': 0.87272,
 'tfidf_vect_n12_ner': 0.86644}

In [49]:
acc, clf, vect = get_accuracy(train_df['review_ner'], train_df['is_positive'], test_df['review_ner'], test_df['is_positive'], LogisticRegression(), TfidfVectorizer(analyzer='char', ngram_range=(2, 6), stop_words = stopword_en, min_df = 0.01))
res['tfidf_vect_nс26_ner'] = acc
res

{'keywords': 0.71224,
 'count_vect': 0.85692,
 'count_vect_lem': 0.85692,
 'tfidf_vect': 0.8666,
 'tfidf_vect_lem': 0.8666,
 'tfidf_vect_n12': 0.86748,
 'tfidf_vect_n13': 0.86748,
 'tfidf_vect_n22': 0.62112,
 'tfidf_vect_nc26': 0.88748,
 'tfidf_vect_nc26_lem': 0.87372,
 'tfidf_vect_stem': 0.86948,
 'tfidf_vect_n12_stem': 0.87136,
 'tfidf_vect_nс26_stem': 0.87272,
 'tfidf_vect_n12_ner': 0.86644,
 'tfidf_vect_nс26_ner': 0.8856}

In [50]:
pd.DataFrame.from_dict(res, orient='index', columns = ['accuracy']).sort_values(by = 'accuracy', ascending = False)

,accuracy
tfidf_vect_nc26,0.88748
tfidf_vect_nс26_ner,0.88560
tfidf_vect_nc26_lem,0.87372
tfidf_vect_nс26_stem,0.87272
tfidf_vect_n12_stem,0.87136
tfidf_vect_stem,0.86948
tfidf_vect_n12,0.86748
tfidf_vect_n13,0.86748
tfidf_vect,0.86660
tfidf_vect_lem,0.86660


Самые высокие результаты у tfidf по символам, замена имен на тэг дала достаточно неплохой результат

#### neural net

In [51]:
train_df = pd.read_csv("train_processed.csv", index_col = 0, encoding = 'utf8')
test_df = pd.read_csv("test_processed.csv", index_col = 0, encoding = 'utf8')

In [52]:
max_len = max([len(s.split()) for s in train_df.review])
max_len

2450

In [53]:
words_counter = Counter((word for text in train_df.review for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

Words count 20923


In [54]:
words_counter.most_common()

[('the', 331886),
 ('and', 162232),
 ('a', 161580),
 ('of', 145327),
 ('to', 135064),
 ('is', 106975),
 ('in', 92987),
 ('it', 77816),
 ('i', 75260),
 ('this', 75246),
 ('that', 69462),
 ('was', 48090),
 ('as', 46645),
 ('for', 43993),
 ('with', 43929),
 ('movie', 42407),
 ('but', 41728),
 ('film', 38092),
 ('on', 33596),
 ('not', 30226),
 ('are', 29337),
 ('you', 29326),
 ('his', 29249),
 ('have', 27682),
 ('be', 26680),
 ('he', 26613),
 ('one', 25626),
 ('its', 24919),
 ('at', 23351),
 ('all', 23275),
 ('by', 22358),
 ('an', 21456),
 ('they', 20902),
 ('from', 20408),
 ('who', 20299),
 ('so', 19863),
 ('like', 19644),
 ('her', 18242),
 ('or', 17607),
 ('just', 17571),
 ('about', 17302),
 ('has', 16724),
 ('if', 16510),
 ('out', 16431),
 ('some', 15667),
 ('there', 15444),
 ('what', 14990),
 ('good', 14504),
 ('more', 14145),
 ('when', 14053),
 ('very', 13980),
 ('even', 12503),
 ('she', 12458),
 ('my', 12308),
 ('up', 12222),
 ('no', 12218),
 ('would', 12134),
 ('time', 12035),
 ('wh

In [55]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

X_train = convert(train_df.review, word2idx, max_len)
X_test = convert(test_df.review, word2idx, max_len)

In [56]:
X_train.shape[1]

2450

In [69]:
model = Sequential([
    Embedding(input_dim=len(word2idx), output_dim=128, input_length=X_train.shape[1]),
    
    GlobalMaxPooling1D(),
    Dense(units=10, activation='relu'),
    Dense(units=10, activation='relu'),
    
    Dense(units=1, activation='sigmoid')
])

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2450, 128)         2678144   
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 2,679,555
Trainable params: 2,679,555
Non-trainable params: 0
_________________________________________________________________


In [70]:
tf.debugging.set_log_device_placement(True)
model.fit(X_train, train_df.is_positive, batch_size=128, epochs=4)

Epoch 1/4
196/196 [==============================] - 25s 126ms/step - loss: 0.5667 - accuracy: 0.7650
Epoch 2/4
196/196 [==============================] - 25s 128ms/step - loss: 0.2728 - accuracy: 0.8892
Epoch 3/4
196/196 [==============================] - 25s 127ms/step - loss: 0.1770 - accuracy: 0.9354
Epoch 4/4
196/196 [==============================] - 25s 128ms/step - loss: 0.1062 - accuracy: 0.9669


In [72]:
res['nn'] = model.evaluate(X_test, test_df.is_positive, batch_size = 128)[1]

196/196 [==============================] - 9s 45ms/step - loss: 0.3153 - accuracy: 0.8772


In [73]:
pd.DataFrame.from_dict(res, orient='index', columns = ['accuracy']).sort_values(by = 'accuracy', ascending = False)

,accuracy
tfidf_vect_nc26,0.88748
tfidf_vect_nс26_ner,0.88560
nn,0.87724
tfidf_vect_nc26_lem,0.87372
tfidf_vect_nс26_stem,0.87272
tfidf_vect_n12_stem,0.87136
tfidf_vect_stem,0.86948
tfidf_vect_n12,0.86748
tfidf_vect_n13,0.86748
tfidf_vect,0.86660


Сетка оказалась не самым лучшим вариантом по точности

In [ ]:
# model2 = Sequential([
#     Embedding(input_dim=len(word2idx), output_dim=128, input_length=X_train.shape[1]),
#     LSTM(units = 128, return_sequences=True),
    
#     Dense(units=1, activation='sigmoid')
# ])

# model2.summary()
# model2.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

In [ ]:
# tf.debugging.set_log_device_placement(True)
# model2.fit(X_train, train_df.is_positive, batch_size=128, epochs=10)

In [ ]:
# model2.evaluate(X_test, test_df.is_positive, batch_size = 16)[1]